In [47]:
import os
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
import mlflow
from mlflow.tracking import MlflowClient

In [48]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

# Extract the path part from the tracking URI
db_path = MLFLOW_TRACKING_URI.replace("sqlite:///", "")

# Get the absolute path
absolute_db_path = os.path.abspath(db_path)
absolute_tracking_uri = f"sqlite:///{absolute_db_path}"

# Print the absolute path of the tracking URI
print("Absolute Tracking URI:", absolute_tracking_uri)

Absolute Tracking URI: sqlite:////Users/nikolaykozlovskiy/Documents/mlops-zoomcamp_DT/02-experiment-tracking/mlflow.db


In [13]:
def print_run_info(runs):
    for r in runs:
        print("run_id: {}".format(r.info.run_id))
        print("lifecycle_stage: {}".format(r.info.lifecycle_stage))
        print("metrics: {}".format(r.data.metrics))

        # Exclude mlflow system tags
        tags = {k: v for k, v in r.data.tags.items() if not k.startswith("mlflow.")}
        print("tags: {}".format(tags))

In [44]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
experiment_id = '1'
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [45]:
client.create_experiment("test_2")

'2'

In [46]:
for experiment in client.search_experiments(): 
    print(f'{experiment.experiment_id}:{experiment.name}')

2:test_2
1:nyc-taxi-experiment
0:Default


In [18]:
runs = client.search_runs(
    experiment_id, 
    run_view_type=mlflow.entities.ViewType.ALL
)
print_run_info(runs)

run_id: fb9fcfdb58984dc5aff61996d8a38237
lifecycle_stage: active
metrics: {'rmse': 5.176028795790886}
tags: {'developer': 'nikolay', 'model': 'xgboost'}
run_id: c12c2aa17f8d44018a0f5b172538a947
lifecycle_stage: active
metrics: {'rmse': 6.682970148789025}
tags: {'developer': 'nikolay', 'model': 'xgboost'}
run_id: 976d9123c3ef4efc818fd9d32983a159
lifecycle_stage: deleted
metrics: {}
tags: {'developer': 'nikolay', 'model': 'xgboost'}
run_id: 935d608fb2514f5fb03a6fac90c47d4f
lifecycle_stage: active
metrics: {'rmse': 9.242491841499588}
tags: {'developer': 'nikolay'}


In [19]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/nikolaykozlovskiy/Documents/mlops-zoomcamp_DT/02-experiment-tracking/mlruns/1', creation_time=1716664218811, experiment_id='1', last_update_time=1716664218811, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [20]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [21]:
df_train = read_dataframe('../data/green_tripdata_2023-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2023-02.parquet')

In [22]:
len(df_train), len(df_val)

(65946, 62574)

In [25]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [26]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [27]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [28]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/opt/homebrew/anaconda3/envs/mlops_zoomcamp_env/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


6.037547854084554

In [23]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [31]:
with mlflow.start_run():

    mlflow.set_tag("developer", "nikolay")

    mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "../data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

/opt/homebrew/anaconda3/envs/mlops_zoomcamp_env/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [49]:
import xgboost as xgb

In [50]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [51]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [52]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("developer", "nikolay")
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=10,
            evals=[(valid, 'validation')],
            early_stopping_rounds=5
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [53]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=30,
    trials=Trials()
)

  0%|          | 0/30 [00:00<?, ?trial/s, best loss=?]

/opt/homebrew/anaconda3/envs/mlops_zoomcamp_env/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [17:39:29] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.29959                           
[1]	validation-rmse:6.21780                           
[2]	validation-rmse:5.68107                           
[3]	validation-rmse:5.42287                           
[4]	validation-rmse:5.30145                           
[5]	validation-rmse:5.23982                           
[6]	validation-rmse:5.21370                           
[7]	validation-rmse:5.19944                           
[8]	validation-rmse:5.19059                           
[9]	validation-rmse:5.18544                           
[0]	validation-rmse:8.18252                                                    
[1]	validation-rmse:7.33988                                                    
[2]	validation-rmse:6.73149                                                    
[3]	validation-rmse:6.29780                                                    
  3%|▎         | 1/30 [00:01<00:38,  1.34s/trial, best loss: 5.185443513807581]

/opt/homebrew/anaconda3/envs/mlops_zoomcamp_env/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/opt/homebrew/anaconda3/envs/mlops_zoomcamp_env/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [17:39:30] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[4]	validation-rmse:5.99134                                                    
[5]	validation-rmse:5.77424                                                    
[6]	validation-rmse:5.62423                                                    
[7]	validation-rmse:5.51945                                                    
[8]	validation-rmse:5.44585                                                    
[9]	validation-rmse:5.39164                                                    
  7%|▋         | 2/30 [00:01<00:21,  1.28trial/s, best loss: 5.185443513807581]

...

/opt/homebrew/anaconda3/envs/mlops_zoomcamp_env/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/opt/homebrew/anaconda3/envs/mlops_zoomcamp_env/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [17:39:56] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:6.71997                                                     
[2]	validation-rmse:6.09203                                                     
[3]	validation-rmse:5.71656                                                     
[4]	validation-rmse:5.49486                                                     
[5]	validation-rmse:5.36456                                                     
[6]	validation-rmse:5.28911                                                     
[7]	validation-rmse:5.24483                                                     
[8]	validation-rmse:5.21390                                                     
[9]	validation-rmse:5.19733                                                     
[0]	validation-rmse:7.47582                                                     
 97%|█████████▋| 29/30 [00:28<00:00,  1.20trial/s, best loss: 5.177241217125009]

/opt/homebrew/anaconda3/envs/mlops_zoomcamp_env/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/opt/homebrew/anaconda3/envs/mlops_zoomcamp_env/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [17:39:57] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:6.44086                                                     
[2]	validation-rmse:5.88997                                                     
[3]	validation-rmse:5.60508                                                     
[4]	validation-rmse:5.45728                                                     
[5]	validation-rmse:5.37785                                                     
[6]	validation-rmse:5.33510                                                     
[7]	validation-rmse:5.30739                                                     
[8]	validation-rmse:5.29299                                                     
[9]	validation-rmse:5.27985                                                     
100%|██████████| 30/30 [00:29<00:00,  1.03trial/s, best loss: 5.177241217125009]


/opt/homebrew/anaconda3/envs/mlops_zoomcamp_env/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [35]:
mlflow.xgboost.autolog(disable=True)

In [36]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=30,
        evals=[(valid, 'validation')],
        early_stopping_rounds=20
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:8.73788
[1]	validation-rmse:8.22960


/opt/homebrew/anaconda3/envs/mlops_zoomcamp_env/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [16:03:20] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[2]	validation-rmse:7.78914
[3]	validation-rmse:7.40823
[4]	validation-rmse:7.08398
[5]	validation-rmse:6.80130
[6]	validation-rmse:6.56559
[7]	validation-rmse:6.35942
[8]	validation-rmse:6.18716
[9]	validation-rmse:6.04364
[10]	validation-rmse:5.91994
[11]	validation-rmse:5.81441
[12]	validation-rmse:5.72701
[13]	validation-rmse:5.65236
[14]	validation-rmse:5.58821
[15]	validation-rmse:5.53629
[16]	validation-rmse:5.49451
[17]	validation-rmse:5.45443
[18]	validation-rmse:5.42147
[19]	validation-rmse:5.39347
[20]	validation-rmse:5.37267
[21]	validation-rmse:5.35128
[22]	validation-rmse:5.33257
[23]	validation-rmse:5.31780
[24]	validation-rmse:5.30449
[25]	validation-rmse:5.29542
[26]	validation-rmse:5.28365
[27]	validation-rmse:5.27505
[28]	validation-rmse:5.26725
[29]	validation-rmse:5.26004


/opt/homebrew/anaconda3/envs/mlops_zoomcamp_env/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/homebrew/anaconda3/envs/mlops_zoomcamp_env/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [16:03:22] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "../data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
